<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Sizing of a multi-rotor drone

*Written by Marc Budinger, Aitor Ochotorena (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

The objective of this notebook is to select the best compromise of components (propeller, motor, ESC, battery) of a multi-rotor drone for given specifiations.

**Scipy** and **math** packages will be used for this notebook in order to illustrate the optimization algorithms of python.

In [1]:
import os.path as pth
import scipy
import scipy.optimize
from math import pi
from math import sqrt
from math import sin
import math
import numpy as np
import timeit
import pandas as pd
from IPython.display import display, HTML

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
from utils.model_serializer import ModelSerializer

pd.options.display.float_format = '{:,.2f}'.format

## Sizing code

The set of equations of a sizing code can generate typical issues such : 
- Underconstrained set of equations: the lacking equations can come from additional scenarios, estimation models or additional sizing variable.   
- overconstrained equations often due to the selection of a component on multiple critera: the adding of over-sizing coefficients and constraints in the optimization problem can generally fix this issue   
- algebraic loops often due to selection criteria requiring informations generally available after the selection 

Concerning overconstraints components, we have here:
- Brushless motors with multiple torque and voltage constraints (hover and transient vertical displacement) 

Multiple algebraic loops appears in the sizing problem:
- The thrust depends of the total mass which depend of components required for generating this thrust

The final optimization problem depends thus of these parameters:
- $\beta_{pro}=pitch/diameter$ ratio to define the propeller
- $k_{os}$ over sizing coefficient on the load mass to estimate the final total mass
- $k_{mot}$ over sizing coeffcient on the motor torque to estimate the max torque with the hover flight conditions
- $k_{speed,mot}$ over sizing coeffcient on the motor speed to take into account voltage limits during hover or take-off flight
- $k_{ND}$ slow down propeller coef : ND = kNDmax / k_ND
- $k_{D}$ aspect ratio e_arm/D_out_arm (thickness/diameter) for the beam of the frame
- $k_{mb}$ ratio battery mass / payload mass
- $k_{vb}$ over sizing coefficient for the battery voltage

More details in the setting up of sizing code can be found in the  [following paper](https://www.researchgate.net/profile/Marc_Budinger/publication/277933677_Computer-aided_definition_of_sizing_procedures_and_optimization_problems_of_mechatronic_systems/links/55969de508ae793d137c7ea5/Computer-aided-definition-of-sizing-procedures-and-optimization-problems-of-mechatronic-systems.pdf):  

> Reysset, A., Budinger, M., & Maré, J. C. (2015). Computer-aided definition of sizing procedures and optimization problems of mechatronic systems. Concurrent Engineering, 23(4), 320-332.

The sizing code is defined here in a function which can give:
- an evaluation of the objective: here the total mass
- an evaluation of the constraints: 

Here is an non-exhaustive XDSM diagram of the multirotor sizing code:

![XDSM](pictures/xdsm_multirotor_mdo.png)

## Objectives and specifications

Main specifications :
- a load (video, control card) of mass $M_{load}$.  
- an autonomy $t_{hf}$ for the hover flight.
- an acceleration to take off $a_{to}$.


In [2]:
# Specifications

# Load
M_pay = 100. # [kg] load mass

# Acceleration during take off
a_to = 0.25 * 9.81 # [m/s²] acceleration

# Autonomy
t_hov_spec = 25. # [min] time of hover flight

# MTOW
MTOW = 360. # [kg] maximal mass

# Objectif
MAX_TIME = False # Objective

## Architecture definition and design assumptions

In [3]:
# Architecture of the multi-rotor drone (4,6, 8 arms, ...)
N_arm = 4 # [-] number of arm
N_pro_arm = 2 # [-] number of propeller per arm (1 or 2)
N_pro = N_pro_arm * N_arm # [-] Propellers number

## Reference parameters

- ### Battery & ESC

In [4]:
# Reference parameters for scaling laws
# Ref : MK-quadro
M_bat_ref = .329 # [kg] mass
E_bat_ref = 220.*3600.*.329 # [J]

# Ref : Turnigy K_Force 70HV 
P_esc_ref = 3108. # [W] Power
M_esc_ref = .115 # [kg] Mass

- ### Motor

In [5]:
# Motor reference
# Ref : AXI 5325/16 GOLD LINE
T_nom_mot_ref = 2.32  # [N.m] rated torque
T_max_mot_ref = 85./70.*T_nom_mot_ref # [N.m] max torque
R_mot_ref = 0.03  # [Ohm] resistance
M_mot_ref = 0.575 # [kg] mass
K_mot_ref = 0.03 # [N.m/A] torque coefficient
T_mot_fr_ref = 0.03 # [N.m] friction torque (zero load, nominal speed)


- ### Frame

In [6]:
# Reference parameters for scaling laws
sigma_max = 280e6/4. # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
rho_s = 1700. # [kg/m3] Volumic mass of aluminum

- ### Propeller

In [7]:
# Specifications
rho_air=1.18# [kg/m^3] Air density
ND_max=105000./60.*.0254 #[Hz.m] Max speed limit (N.D max) for APC MR propellers


# Reference parameters for scaling laws
D_pro_ref=11.*.0254# [m] Reference propeller diameter
M_pro_ref=0.53*0.0283# [kg] Reference propeller mass

In [8]:
serializer = ModelSerializer()
model_folder = 'models'
file_name = 'sizing_code'
file_path = pth.join(model_folder, file_name)
sizing_code = serializer.load_model(file_path)

## Optimization problem


We will now use the [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLSQP algorithm without explicit expression of the gradient (Jacobian). A course on Multidisplinary Gradient optimization algorithms and gradient optimization algorithm is given [here](http://mdolab.engin.umich.edu/sites/default/files/Martins-MDO-course-notes.pdf):
> Joaquim R. R. A. Martins (2012). A Short Course on Multidisciplinary Design Optimization. University of Michigan


The first step is to give an initial value of optimisation variables:

In [9]:
# Optimisation variables
beta_pro = .33 # pitch/diameter ratio of the propeller
k_os = 3.2 # over sizing coefficient on the load mass 
k_ND = 1.2 # slow down propeller coef : ND = kNDmax / k_ND
k_mot = 1. # over sizing coefficient on the motor torque
k_speed_mot = 1.2 # adaption winding coef on the motor speed
k_mb = 1. # ratio battery/load mass
k_vb = 1. # oversizing coefficient for voltage evaluation
k_D = .01 # aspect ratio e/c (thickness/side) for the beam of the frame


# Vector of parameters
parameters = scipy.array((beta_pro, k_os, k_ND,k_mot, k_speed_mot, k_mb, k_vb,k_D,))

d:\users\s.delbecq\appdata\local\programs\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  del sys.path[0]


We can print of the characterisitcs of the problem before optimization with the initial vector of optimization variables:

In [10]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")
sizing_code(parameters,'Prt')
print("-----------------------------------------------")

-----------------------------------------------
Initial characteristics before optimization :


interactive(children=(Dropdown(description='Type', options=('Battery & ESC', 'Constraints', 'Frame', 'Motor', …

-----------------------------------------------


Then we can solve the problem and print of the optimized solution:

In [11]:
# Optimization with SLSQP algorithm
contrainte = lambda x: sizing_code(x, 'Const')
objectif = lambda x: sizing_code(x, 'Obj')
objectifP = lambda x: sizing_code(x, 'ObjP')

SLSQP = False # Optimization algorithm choice

# Optimization bounds
# beta,  k_os, k_ND, k_mot, k_speed_mot, k_mb, k_vb, k_D
bounds = [(0.3,0.6), (1,400), (1,100), (1,100), (1,400), (0.1,100), (1,5), (0.1,0.99)]

if SLSQP == True:
    # SLSQP omptimisation
    result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)
else:
    # Differential evolution omptimisation
    result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=bounds,
                                   tol=1e-12)

# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

if SLSQP == True:
    sizing_code(result,'Obj')
    sizing_code(result, 'Prt')
else:
    sizing_code(result.x,'Obj')
    sizing_code(result.x, 'Prt')
print("-----------------------------------------------")

-----------------------------------------------
Final characteristics after optimization :


interactive(children=(Dropdown(description='Type', options=('Battery & ESC', 'Constraints', 'Frame', 'Motor', …

-----------------------------------------------
